# Installation des librairies et vérification que CUDA est bien installé

In [1]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

AttributeError: module 'keras.backend' has no attribute 'tensorflow_backend'

In [ ]:
!pip install keras
!pip install scipy
!pip install pillow
!pip uninstall tensorflow



### Vérification des GPUs disponible

In [3]:
import tensorflow as tf
print("GPUs Available: ", tf.config.list_physical_devices('GPU'))


GPUs Available:  []


## 1. Préparation des Données
### 1.1 Organisation des Images : Rangez vos images dans une structure de dossiers appropriée, typiquement un dossier pour chaque classe (homme, femme).

In [ ]:
# 1. Préparation des Données
# Organisation des Images : Rangez vos images dans une structure de dossiers appropriée, typiquement un dossier pour chaque classe (homme, femme).

# classifier.py
# Permet de classifier les images (homme ou femme) afin de créer des données.


import json
import os
from pathlib import Path
from PIL import Image
import cv2 as cv


# Chemin vers votre fichier JSON
fichier_json = '../Data/sorted_by_label.json'

# Charger les vecteurs moyens à partir du fichier JSON
with open(fichier_json, 'r') as file:
    json_content = json.load(file)

# Path to the directory where images are stored
image_directory_path = Path('../Image/img_align_celeba/img_align_celeba')

# Function to display an image by filename
def display_image(filename):
    try:
        image_path = image_directory_path / filename
        if not image_path.exists():
            print(f"Image file does not exist: {image_path}")
            return
        image = Image.open(image_path)
        image.show()
    except IOError as e:
        print(f"Error opening image {filename}: {e}")


# Function to classify images based on user input and create a JSON structure
def classify_images(json_data, max_labels=None):
    classification_result = {'men': [], 'women': []}
    label_count = 0

    for label, images in json_data['data'].items():
        if max_labels is not None and label_count >= max_labels:
            break  # Arrête le traitement après un certain nombre de labels

        if images:  # If there are images for the label
            # Display the first image of the label
            print(images[0])
            display_image(images[0])
            # Ask the user to classify the label
            classification = input("Classify the person as a man (m) or a woman (w): ").strip().lower()
            if classification == 'm':
                classification_result['men'].extend(images)
            elif classification == 'w':
                classification_result['women'].extend(images)
            else:
                print("Invalid input, skipping this label.")

        label_count += 1

    # Write the classification result to a JSON file
    with open('classification_result.json', 'w') as outfile:
        json.dump(classification_result, outfile, indent=4)

    print("Classification complete! Results saved to classification_result.json.")


# Exemple d'utilisation : traiter seulement les 100 premiers labels
classify_images(json_content, max_labels=100)

In [ ]:
# trierHommeFemme.py
# Permet de prendre un fichier json en entré comprennant deux labels (ex: homme et femme), puis il va trier les images dans les bons répertoires.

import json
import os
import shutil

# Chemin du fichier JSON
chemin_fichier_json = 'classification_result-testSet.json'

# Chemins des dossiers source et cible
chemin_dossier_source = '../Image/img_align_celeba/img_align_celeba/'
chemin_dossier_hommes = '../Image/testSet/hommes/'
chemin_dossier_femmes = '../Image/testSet/femmes/'

# Créer les dossiers cibles s'ils n'existent pas
os.makedirs(chemin_dossier_hommes, exist_ok=True)
os.makedirs(chemin_dossier_femmes, exist_ok=True)

# Charger les données JSON
with open(chemin_fichier_json, 'r') as fichier:
    data = json.load(fichier)

# Déplacer les images dans les dossiers correspondants
for homme in data['men']:
    chemin_source = os.path.join(chemin_dossier_source, homme)
    chemin_destination = os.path.join(chemin_dossier_hommes, homme)
    shutil.move(chemin_source, chemin_destination)

for femme in data['women']:
    chemin_source = os.path.join(chemin_dossier_source, femme)
    chemin_destination = os.path.join(chemin_dossier_femmes, femme)
    shutil.move(chemin_source, chemin_destination)


## 1. Préparation des Données

### 1.2 Augmentation des Données : Pour éviter le surapprentissage et améliorer la généralisation, surtout si vous disposez de peu de données, utilisez des techniques d'augmentation d'images (comme la rotation, le zoom, le décalage horizontal/vertical, etc.).

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Configuration de l'augmentation des données
datagen = ImageDataGenerator(
    rescale=1./255,         # Normalisation des valeurs de pixels
    rotation_range=40,      # Rotation aléatoire de l'image (degrés, 0-180)
    width_shift_range=0.2,  # Décalage horizontal aléatoire (fraction de la largeur totale)
    height_shift_range=0.2, # Décalage vertical aléatoire (fraction de la hauteur totale)
    shear_range=0.2,        # Cisaillement aléatoire
    zoom_range=0.2,         # Zoom aléatoire à l'intérieur des images
    horizontal_flip=True,   # Retournement aléatoire des images horizontalement
    fill_mode='nearest'     # Stratégie pour remplir les pixels nouvellement créés
)

# Chemin vers le dossier d'entraînement
train_dir = '../Image/trainSet/'  # Mettez à jour avec le chemin approprié

# Générateur de données d'entraînement
train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(178, 218),  # Taille des images après redimensionnement
    batch_size=32,
    class_mode='binary'  # 'binary' pour classification binaire, 'categorical' pour multiclasse
)

# Chemin vers le dossier de validation
validation_dir = '../Image/validationSet/'  # Mettez à jour avec le chemin approprié

# Générateur de données de validation
validation_generator = datagen.flow_from_directory(
    validation_dir,
    target_size=(178, 218),  # Taille des images après redimensionnement
    batch_size=32,
    class_mode='binary'  # 'binary' pour classification binaire, 'categorical' pour multiclasse
)

# Chemin vers le dossier de test
test_dir = '../Image/testSet/'  # Mettez à jour avec le chemin approprié

# Générateur de données de test
test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(178, 218),  # Taille des images après redimensionnement
    batch_size=32,
    class_mode='binary'  # 'binary' pour classification binaire, 'categorical' pour multiclasse
)


## 2. Construction du Modèle
### 2.1 - Choix du Modèle : Vous pouvez construire un modèle de classification d'images à partir de zéro ou utiliser le transfert d'apprentissage à partir d'un modèle pré-entraîné.
### 2.2 - Architecture du Modèle : Définissez l'architecture de votre modèle (couches, neurones, fonctions d'activation, etc.).
### 2.3 - Compilation du Modèle : Compilez le modèle avec un optimiseur approprié, une fonction de perte et des métriques.


In [ ]:
# model-perso.py
# 2.1) Création d'un model à partir de zéro

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

### 2.2 - Architecture du Modèle
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(178, 218, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # 1 pour classification binaire (homme/femme)
])

### 2.3 - Compilation du Modèle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model-précrée.py

# # Utilisation d'un model déjà existant, on vient juste rajouter des couches
# 
# from keras.applications.vgg16 import VGG16
# from keras.models import Model
# from keras.layers import Dense, GlobalAveragePooling2D
# 
# # Charger VGG16 sans la partie supérieure (top) - sans les couches de classification
# base_model = VGG16(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
# 
# # Ajouter vos propres couches pour la classification
# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# x = Dense(1024, activation='relu')(x)
# predictions = Dense(1, activation='sigmoid')(x)  # 1 pour classification binaire
# 
# model = Model(inputs=base_model.input, outputs=predictions)
# 
# # Compiler le modèle
# for layer in base_model.layers:
#     layer.trainable = False  # Geler les couches du modèle pré-entraîné
# 
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


## 3. Entraînement du Modèle
### 3.1 - Entraînement sur les Données : Entraînez votre modèle sur les données préparées, en utilisant éventuellement la validation croisée pour évaluer sa performance.


In [ ]:
# 3.1) Entraînement sur les données
history = model.fit(
    train_generator,
    steps_per_epoch=69,  # Nombre de lots à utiliser pendant chaque époque 
    # si vous avez 2205 images et un batch_size de 32, vous auriez environ 69 steps par époque (2205 / 32 = 68.9)
    epochs=15,           # Nombre d'époques pour l'entraînement
    # Si vous avez spécifié 100 steps_per_epoch et que vous voulez entraîner le modèle pour 15 époques, votre générateur doit pouvoir fournir (100 * 15 = 1500) lots au total.
    validation_data=validation_generator,  # Si vous avez un générateur de validation
    validation_steps=50   # Nombre de lots à utiliser pour la validation
)

### 3.2 - Callbacks : Utilisez des callbacks pour des fonctionnalités telles que l'enregistrement des meilleures performances, la réduction du taux d'apprentissage, etc.

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

# Enregistrement du meilleur modèle
checkpoint = ModelCheckpoint(
    'meilleur_modele.h5',  # Nom du fichier pour sauvegarder le modèle
    monitor='val_accuracy', # Métrique à surveiller
    verbose=1,             # Afficher des messages détaillés
    save_best_only=True,   # Sauvegarder uniquement le meilleur modèle
    mode='max'             # Mode 'max' pour la métrique 'accuracy'
)

# Arrêt anticipé
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10            # Nombre d'époques sans amélioration après lesquelles l'entraînement est arrêté
)

# Réduction du taux d'apprentissage
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,            # Facteur de réduction du taux d'apprentissage
    patience=5,            # Nombre d'époques sans amélioration avant la réduction
    min_lr=0.001           # Taux d'apprentissage minimal
)

history = model.fit(
    train_generator,
    steps_per_epoch=69,
    epochs=15,
    validation_data=validation_generator,
    validation_steps=50,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)


## 4. Évaluation et Ajustement
### 4.1 - Évaluer la Performance : Évaluez les performances de votre modèle sur un ensemble de données de test pour vérifier sa généralisation.



In [ ]:
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}, Test loss: {test_loss:.4f}")

### 4.2 - Ajustement des Hyperparamètres : Ajustez les hyperparamètres au besoin pour améliorer les performances.

Si les performances de votre modèle sur l'ensemble de test ne sont pas satisfaisantes, vous pourriez avoir besoin d'ajuster les hyperparamètres. Les hyperparamètres sont des paramètres qui ne sont pas appris pendant l'entraînement mais qui sont définis à l'avance et qui influencent le comportement et la performance du modèle d'apprentissage. Voici quelques hyperparamètres courants que vous pouvez ajuster :

- **Taux d'apprentissage** : C'est un des hyperparamètres les plus importants. S'il est trop élevé, l'entraînement peut ne pas converger ; s'il est trop bas, l'entraînement peut être très lent ou se coincer dans un minimum local.
- **Taille des lots** (batch size) : Une plus grande taille de lot peut conduire à une convergence plus stable et plus rapide, mais aussi à une généralisation moins bonne et à des exigences de mémoire plus élevées.
- **Architecture du modèle** : Ajouter ou supprimer des couches, changer le nombre de neurones, essayer différentes fonctions d'activation, etc.
- **Régularisation** : Techniques comme dropout ou L1/L2 pour prévenir le surajustement.
- **Optimiseur** : Changer l'optimiseur (Adam, SGD, RMSprop, etc.) peut affecter la vitesse et la qualité de l'apprentissage.

Après avoir ajusté les hyperparamètres, vous devrez réentraîner votre modèle et le réévaluer pour voir si les performances se sont améliorées.

N'oubliez pas de noter les performances de chaque configuration pour pouvoir comparer et choisir la meilleure. Des outils comme TensorBoard peuvent être très utiles pour suivre les performances et les hyperparamètres pendant l'entraînement.

## 5. Utilisation du Modèle
### 5.1 - Sauvegarde du Modèle : Sauvegardez votre modèle entraîné pour une utilisation future.

In [ ]:
# Sauvegarde du modèle complet
model.save('mon_modele.h5')  # Crée un fichier HDF5 'mon_modele.h5'

In [4]:
import numpy as np
from keras.preprocessing import image
from keras.models import load_model
import json


# Chargement du même modèle
model = load_model('mon_modele.h5')

# Liste pour stocker les résultats
results = []

# Chargement et prétraitement d'une image
img_path = "../Image/2024_ldiraiso.jpg"
img = image.load_img(img_path, target_size=(178, 218))
img_tensor = image.img_to_array(img)  # Convertit l'image en tableau numpy
img_tensor = np.expand_dims(img_tensor, axis=0)  # Ajoute une dimension pour le lot
img_tensor /= 255.  # Normalisation si nécessaire (comme pendant l'entraînement)

# Faire la prédiction
predictions = model.predict(img_tensor)

# Traiter les prédictions
if predictions[0] > 0.75 :
    predicted_class = 'men'
    print(predicted_class, ", with ", predictions[0][0]*100, "% probability")
else :
    predicted_class = 'women'
    print(predicted_class, ", with ", 100 - predictions[0][0]*100, "% probability")

# Fonction pour charger les résultats existants
def load_existing_results(filename):
    try:
        with open(filename, 'r') as f:
            return json.load(f)
    except FileNotFoundError:
        return []  # Retourne une liste vide si le fichier n'existe pas
    
# Fonction pour vérifier si un résultat existe déjà et le mettre à jour
def update_or_add_prediction(filename, predicted_class, confidence, results):
    # Recherche d'un résultat existant avec le même nom de fichier
    for result in results:
        if result['Filename'] == filename:
            # Mise à jour du résultat existant
            result['PredictedClass'] = predicted_class
            result['Confidence'] = confidence
            return
    # Ajout d'un nouveau résultat si aucun résultat existant n'a été trouvé
    results.append({
        "Filename": filename,
        "PredictedClass": predicted_class,
        "Confidence": confidence
    })
    
# Charger les résultats existants
results_file = 'prediction_model1.json'
existing_results = load_existing_results(results_file)

# Mettre à jour ou ajouter un résultat de prédiction
update_or_add_prediction(img_path, predicted_class, float(predictions[0][0]), existing_results)

# Enregistrement dans un fichier JSON avec les résultats mis à jour
with open(results_file, 'w') as f:
    json.dump(existing_results, f, indent=4)


1/1 [==============================] - 0s 68ms/step
women , with  99.99974210877554 % probability


Plus le résultat se rapproche de 100 %, plus la probabilité que ce soit un homme est élevé. 
À l'inverse, plus le résultat se rapproche de 0%, plus la probabilité d'être un homme est faible, et donc celle d'être une femme est élevé.

Afin de mieux présenter le résultat, je soustrais le résultat à 100 dans le cas où le programme détecterait que c'est une femme, ce qui me donne aussi un pourcentage de certitude.

### 5.2 - Déploiement du Modèle : Déployez le modèle pour des prédictions en temps réel ou sur de nouvelles données.

- **Application Web** : Nous avons créé une application en React afin d'afficher nos résultats et pouvoir les comparer plus facilement. 

Amélioration : On pourrait améliorer l'application afin d'afficher différents schémas qui nous aideraient à évaluer le modèle.
